<a id='top'></a>
### Table of contents

1. Importing Visualization Libraries and Data

2. [Deriving new variables](#variables)

- [Affordability](#afford)
- [Percent of undernourished people](#percent)
- [Price index](#index)
- [Food basket size](#basket)
- [Annual Price Growth](#growth)


In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from scipy import stats
from scipy.stats import pearsonr
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import scale
from pylab import rcParams
import matplotlib.ticker as mt
from matplotlib.ticker import ScalarFormatter
from fuzzywuzzy import process, fuzz
from datetime import date
from datetime import datetime

In [2]:
%matplotlib inline
rcParams['figure.figsize']=14,7
sns.set_style('whitegrid')

In [3]:
path=r'C:\Users\frauz\Documents\Python Projects\Final Project\Data\Data Prepared' #creating a path

<a id='variables'></a>
# Deriving new variables

[Back to top](#top)

In [4]:
# Importing the data

df_full=pd.read_pickle(os.path.join(path,'data_market_merged_no_var.pkl'))

df_full.head()

,iso,country,market,latitude,longitude,year_month,pricetype,category,currency,measure_unit,product_name,price_unit,usdprice_unit,date,year,population,millions_undernourished,estim_earnings,inflation,gdp_pcapita
0,AFG,Afghanistan,Fayzabad,37.116638,70.580022,2000-03,Retail,non-food,AFN,L,Diesel,50000.0,1055.0656,2000-03-01,2000,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,Fayzabad,37.116638,70.580022,2000-04,Retail,non-food,AFN,L,Diesel,49000.0,1036.6597,2000-04-01,2000,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,Fayzabad,37.116638,70.580022,2000-05,Retail,"meat, fish and eggs",AFN,Unit,Livestock,1625000.0,34379.0197,2000-05-01,2000,NaN,NaN,NaN,NaN,NaN
3,AFG,Afghanistan,Fayzabad,37.116638,70.580022,2000-05,Retail,non-food,AFN,L,Diesel,44625.0,944.1008,2000-05-01,2000,NaN,NaN,NaN,NaN,NaN
4,AFG,Afghanistan,Fayzabad,37.116638,70.580022,2000-06,Retail,"meat, fish and eggs",AFN,Unit,Livestock,1476000.0,31226.7280,2000-06-01,2000,NaN,NaN,NaN,NaN,NaN


In [5]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2164814 entries, 0 to 2164813
Data columns (total 20 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   iso                      object        
 1   country                  object        
 2   market                   object        
 3   latitude                 float64       
 4   longitude                float64       
 5   year_month               object        
 6   pricetype                object        
 7   category                 object        
 8   currency                 object        
 9   measure_unit             object        
 10  product_name             object        
 11  price_unit               float64       
 12  usdprice_unit            float64       
 13  date                     datetime64[ns]
 14  year                     int32         
 15  population               float64       
 16  millions_undernourished  float64       
 17  estim_earnings           fl

In [6]:
# Visualizations revealed some very suspicious prices. Such in Yemen in 2016, prices in USD were not recorded accurately. 

# I will calculate prices in USD in Yemen in 2016 based on 0.004 exchange rate 

df_full.loc[(df_full['iso']=='YEM')&(df_full['year']==2016), 'usdprice_unit']=df_full['price_unit']*0.004

df_full[['iso','year','price_unit','product_name','usdprice_unit']][(df_full['iso']=='YEM')&(df_full['usdprice_unit']>1000)]

,iso,year,price_unit,product_name,usdprice_unit
2098093,YEM,2022,253250.0,Livestock,1013.2027


In [7]:
df_full[df_full['market'].str.contains('Refugee')]

,iso,country,market,latitude,longitude,year_month,pricetype,category,currency,measure_unit,product_name,price_unit,usdprice_unit,date,year,population,millions_undernourished,estim_earnings,inflation,gdp_pcapita
2155886,ZWE,Zimbabwe,Tongogara Refugee Camp 2,-20.349396,32.308848,2020-04,Retail,cereals and tubers,USD,KG,Rice,1.6333,1.6333,2020-04-01,2020,15669666.0,6.1,NaN,557.201817,1372.696674
2155887,ZWE,Zimbabwe,Tongogara Refugee Camp 2,-20.349396,32.308848,2020-04,Retail,"meat, fish and eggs",USD,KG,Fish,3.2000,3.2000,2020-04-01,2020,15669666.0,6.1,NaN,557.201817,1372.696674
2155888,ZWE,Zimbabwe,Tongogara Refugee Camp 2,-20.349396,32.308848,2020-04,Retail,miscellaneous food,USD,KG,Salt,0.9067,0.9067,2020-04-01,2020,15669666.0,6.1,NaN,557.201817,1372.696674
2155889,ZWE,Zimbabwe,Tongogara Refugee Camp 2,-20.349396,32.308848,2020-04,Retail,miscellaneous food,USD,KG,Sugar,1.7533,1.7533,2020-04-01,2020,15669666.0,6.1,NaN,557.201817,1372.696674
2155890,ZWE,Zimbabwe,Tongogara Refugee Camp 2,-20.349396,32.308848,2020-04,Retail,non-food,USD,KG,Handwash soap,1.4932,1.4932,2020-04-01,2020,15669666.0,6.1,NaN,557.201817,1372.696674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2164723,ZWE,Zimbabwe,Tongogara Refugee Camp 2,-20.349396,32.308848,2022-12,Retail,non-food,USD,KG,Handwash soap,2.4208,2.4208,2022-12-01,2022,NaN,NaN,304.268105,104.705171,1266.996031
2164724,ZWE,Zimbabwe,Tongogara Refugee Camp 2,-20.349396,32.308848,2022-12,Retail,non-food,USD,KG,Laundry soap,2.0922,2.0922,2022-12-01,2022,NaN,NaN,304.268105,104.705171,1266.996031
2164725,ZWE,Zimbabwe,Tongogara Refugee Camp 2,-20.349396,32.308848,2022-12,Retail,non-food,USD,L,Toothpaste,15.3180,15.3180,2022-12-01,2022,NaN,NaN,304.268105,104.705171,1266.996031
2164726,ZWE,Zimbabwe,Tongogara Refugee Camp 2,-20.349396,32.308848,2022-12,Retail,oil and fats,USD,L,Oil,2.8767,2.8767,2022-12-01,2022,NaN,NaN,304.268105,104.705171,1266.996031


<a id='afford'></a>
## Affordability

Product Affordability Index: This index is calculated by dividing the average earnings by the price of the product, both in local currency. It quantifies how many units of a particular product a person can afford based on their monthly income.

[Back to top](#top)

In [59]:
# Calculating monthly product affordability index

df_full['affordability_index']=df_full['estim_earnings']/df_full['price_unit']

In [60]:
df_full['affordability_index'].describe()

count    5.983880e+05
mean     7.362902e+03
std      2.355417e+05
min      1.554059e-01
25%      1.226903e+02
50%      2.518392e+02
75%      5.108692e+02
max      2.217459e+07
Name: affordability_index, dtype: float64

In [61]:
df_full[(df_full['country']=='Timor-Leste')&(df_full['estim_earnings'].notnull())]

,iso,country,market,latitude,longitude,year_month,pricetype,category,currency,measure_unit,...,date,year,population,millions_undernourished,estim_earnings,inflation,gdp_pcapita,affordability_index,%_undernourished,base_price_date
1925696,TLS,Timor-Leste,Balibo,-8.968889,125.040556,2013-01,Retail,cereals and tubers,USD,KG,...,2013-01-01,2013,1161555.0,0.3,412.77,10.987234,1201.423609,565.438356,25.827447,2013-10-01
1925697,TLS,Timor-Leste,Balibo,-8.968889,125.040556,2013-01,Retail,cereals and tubers,USD,KG,...,2013-01-01,2013,1161555.0,0.3,412.77,10.987234,1201.423609,506.466258,25.827447,2013-10-01
1925698,TLS,Timor-Leste,Balibo,-8.968889,125.040556,2013-02,Retail,cereals and tubers,USD,KG,...,2013-02-01,2013,1161555.0,0.3,412.77,10.987234,1201.423609,443.838710,25.827447,2013-10-01
1925699,TLS,Timor-Leste,Balibo,-8.968889,125.040556,2013-02,Retail,cereals and tubers,USD,KG,...,2013-02-01,2013,1161555.0,0.3,412.77,10.987234,1201.423609,506.466258,25.827447,2013-10-01
1925700,TLS,Timor-Leste,Balibo,-8.968889,125.040556,2013-03,Retail,cereals and tubers,USD,KG,...,2013-03-01,2013,1161555.0,0.3,412.77,10.987234,1201.423609,565.438356,25.827447,2013-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1929861,TLS,Timor-Leste,Viqueque,-8.867510,126.365740,2021-12,Retail,oil and fats,USD,L,...,2021-12-01,2021,1320942.0,0.3,257.59,NaN,2741.393945,174.047297,22.711065,2023-06-01
1929862,TLS,Timor-Leste,Viqueque,-8.867510,126.365740,2021-12,Retail,pulses and nuts,USD,KG,...,2021-12-01,2021,1320942.0,0.3,257.59,NaN,2741.393945,75.761765,22.711065,2023-05-01
1929863,TLS,Timor-Leste,Viqueque,-8.867510,126.365740,2021-12,Retail,vegetables and fruits,USD,KG,...,2021-12-01,2021,1320942.0,0.3,257.59,NaN,2741.393945,39.266768,22.711065,2023-06-01
1929864,TLS,Timor-Leste,Viqueque,-8.867510,126.365740,2021-12,Retail,vegetables and fruits,USD,KG,...,2021-12-01,2021,1320942.0,0.3,257.59,NaN,2741.393945,42.506601,22.711065,2023-06-01


<a id='percent'></a>
## Percent of undernourished people

Percent of undernourished people is calculated by deviding number of undernourished people by the total population. Since the number of people is recorded in millions, I first need to multiply it by million.

[Back to top](#top)

In [62]:
# Calculating the percentage of undernourished people by country

df_full['%_undernourished']=df_full['millions_undernourished']*1000000/df_full['population']*100

In [63]:
df_full['%_undernourished'].describe()
# considering that we are focusing on the most vulnerable regions, the numbers look logical

count    1.468366e+06
mean     1.858777e+01
std      1.209146e+01
min      5.669336e-02
25%      7.718678e+00
50%      1.593242e+01
75%      2.766788e+01
max      7.077407e+01
Name: %_undernourished, dtype: float64

<a id='index'></a>
## Price index

Price Index: This value is used for normalized comparison across all prices and currencies. It's calculated by dividing the new price by the base price and then multiplying it by one hundred.

Since different countries have different observation periods, there is no single date when observations for each product across  all countries were made. Therefore, I've decided to use the latest observation for each country are product to determine the base price. 

[Back to top](#top)

In [64]:
# Finding the last (max) observation date for each country, pricetype, product

df_full['base_price_date']=df_full.groupby(['country','market','pricetype','product_name'])['date'].transform(np.max)

In [65]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2164814 entries, 0 to 2164813
Data columns (total 23 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   iso                      object        
 1   country                  object        
 2   market                   object        
 3   latitude                 float64       
 4   longitude                float64       
 5   year_month               object        
 6   pricetype                object        
 7   category                 object        
 8   currency                 object        
 9   measure_unit             object        
 10  product_name             object        
 11  price_unit               float64       
 12  usdprice_unit            float64       
 13  date                     datetime64[ns]
 14  year                     int32         
 15  population               float64       
 16  millions_undernourished  float64       
 17  estim_earnings           fl

In [66]:
df_full[df_full['base_price_date'].isnull()]

,iso,country,market,latitude,longitude,year_month,pricetype,category,currency,measure_unit,...,date,year,population,millions_undernourished,estim_earnings,inflation,gdp_pcapita,affordability_index,%_undernourished,base_price_date


In [67]:
#Isolating records that contain base prices

df_base=df_full.loc[df_full['date']==df_full['base_price_date']]

df_base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53452 entries, 545 to 2164813
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   iso                      53452 non-null  object        
 1   country                  53452 non-null  object        
 2   market                   53452 non-null  object        
 3   latitude                 53452 non-null  float64       
 4   longitude                53452 non-null  float64       
 5   year_month               53452 non-null  object        
 6   pricetype                53452 non-null  object        
 7   category                 53452 non-null  object        
 8   currency                 53452 non-null  object        
 9   measure_unit             53452 non-null  object        
 10  product_name             53452 non-null  object        
 11  price_unit               53452 non-null  float64       
 12  usdprice_unit            534

In [68]:
df_base.duplicated().value_counts()

False    53452
dtype: int64

In [69]:
df_base[['country','market','pricetype','product_name','price_unit']].value_counts()

country      market                   pricetype  product_name  price_unit
Afghanistan  Badakhshan               Retail     Bread         77.000000     1
Philippines  Masbate                  Retail     Squashes      49.380000     1
             Maguindanao              Retail     Garlic        179.130000    1
                                                 Ginger        114.000000    1
                                                 Groundnuts    115.440000    1
                                                                            ..
India        Meerut                   Retail     Wheat         26.000000     1
                                                 Wheat flour   38.600000     1
             Mirzapur (vindhyanchal)  Retail     Ghee          160.330000    1
                                                 Lentils       90.666667     1
Zimbabwe     Zvishavane Urban         Retail     Toothpaste    11.955000     1
Length: 53452, dtype: int64

In [70]:
# Merging the new dataframe with the main one to add base price to each record

df_base_price=df_full.merge(df_base[['country','market','pricetype','product_name','price_unit']], on=['country','market','pricetype','product_name'])

In [71]:
df_base_price.tail(70)

,iso,country,market,latitude,longitude,year_month,pricetype,category,currency,measure_unit,...,year,population,millions_undernourished,estim_earnings,inflation,gdp_pcapita,affordability_index,%_undernourished,base_price_date,price_unit_y
2164744,ZWE,Zimbabwe,Ntunjambili,-20.444799,28.694201,2022-07,Retail,oil and fats,USD,L,...,2022,NaN,NaN,304.268105,104.705171,1266.996031,103.005554,NaN,2022-07-01,2.9539
2164745,ZWE,Zimbabwe,Ntunjambili,-20.444799,28.694201,2022-06,Retail,pulses and nuts,USD,KG,...,2022,NaN,NaN,304.268105,104.705171,1266.996031,59.934230,NaN,2022-07-01,4.1354
2164746,ZWE,Zimbabwe,Ntunjambili,-20.444799,28.694201,2022-07,Retail,pulses and nuts,USD,KG,...,2022,NaN,NaN,304.268105,104.705171,1266.996031,73.576463,NaN,2022-07-01,4.1354
2164747,ZWE,Zimbabwe,Pachanza,-16.473037,31.745454,2022-11,Retail,cereals and tubers,USD,KG,...,2022,NaN,NaN,304.268105,104.705171,1266.996031,584.793589,NaN,2022-11-01,0.5203
2164748,ZWE,Zimbabwe,Pachanza,-16.473037,31.745454,2022-11,Retail,cereals and tubers,USD,KG,...,2022,NaN,NaN,304.268105,104.705171,1266.996031,230.471220,NaN,2022-11-01,1.3202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2164809,ZWE,Zimbabwe,Zimuto Siding,-19.858553,30.859637,2022-07,Retail,oil and fats,USD,L,...,2022,NaN,NaN,304.268105,104.705171,1266.996031,59.332340,NaN,2022-11-01,3.0753
2164810,ZWE,Zimbabwe,Zimuto Siding,-19.858553,30.859637,2022-11,Retail,oil and fats,USD,L,...,2022,NaN,NaN,304.268105,104.705171,1266.996031,98.939324,NaN,2022-11-01,3.0753
2164811,ZWE,Zimbabwe,Zimuto Siding,-19.858553,30.859637,2022-07,Retail,pulses and nuts,USD,KG,...,2022,NaN,NaN,304.268105,104.705171,1266.996031,77.254819,NaN,2022-11-01,2.4851
2164812,ZWE,Zimbabwe,Zimuto Siding,-19.858553,30.859637,2022-11,Retail,pulses and nuts,USD,KG,...,2022,NaN,NaN,304.268105,104.705171,1266.996031,122.436966,NaN,2022-11-01,2.4851


In [72]:
df_base_price.info() #the number of records is the same

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2164814 entries, 0 to 2164813
Data columns (total 24 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   iso                      object        
 1   country                  object        
 2   market                   object        
 3   latitude                 float64       
 4   longitude                float64       
 5   year_month               object        
 6   pricetype                object        
 7   category                 object        
 8   currency                 object        
 9   measure_unit             object        
 10  product_name             object        
 11  price_unit_x             float64       
 12  usdprice_unit            float64       
 13  date                     datetime64[ns]
 14  year                     int32         
 15  population               float64       
 16  millions_undernourished  float64       
 17  estim_earnings           fl

In [73]:
df_base_price.duplicated().value_counts() #no duplicates detected

False    2164814
dtype: int64

In [74]:
# Dropping base_price_date and renaming price_units

df_base_price.drop(columns='base_price_date', inplace=True)
df_base_price.rename(columns={'price_unit_y':'base_price','price_unit_x':'price_unit'}, inplace=True)

In [75]:
# Calculating price_index

df_base_price['price_index']=df_base_price['price_unit']/df_base_price['base_price']*100

In [76]:
df_base_price['price_index'].describe()

count    2.164814e+06
mean     8.702463e+01
std      9.645245e+02
min      3.484199e-03
25%      5.006961e+01
50%      7.647536e+01
75%      1.000000e+02
max      8.000000e+05
Name: price_index, dtype: float64

In [77]:
df_base_price

,iso,country,market,latitude,longitude,year_month,pricetype,category,currency,measure_unit,...,year,population,millions_undernourished,estim_earnings,inflation,gdp_pcapita,affordability_index,%_undernourished,base_price,price_index
0,AFG,Afghanistan,Fayzabad,37.116638,70.580022,2000-03,Retail,non-food,AFN,L,...,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.5000,126582.278481
1,AFG,Afghanistan,Fayzabad,37.116638,70.580022,2000-04,Retail,non-food,AFN,L,...,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.5000,124050.632911
2,AFG,Afghanistan,Fayzabad,37.116638,70.580022,2000-05,Retail,non-food,AFN,L,...,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.5000,112974.683544
3,AFG,Afghanistan,Fayzabad,37.116638,70.580022,2000-06,Retail,non-food,AFN,L,...,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.5000,151392.405063
4,AFG,Afghanistan,Fayzabad,37.116638,70.580022,2000-07,Retail,non-food,AFN,L,...,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.5000,158227.848101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2164809,ZWE,Zimbabwe,Zimuto Siding,-19.858553,30.859637,2022-07,Retail,oil and fats,USD,L,...,2022,NaN,NaN,304.268105,104.705171,1266.996031,59.332340,NaN,3.0753,166.754463
2164810,ZWE,Zimbabwe,Zimuto Siding,-19.858553,30.859637,2022-11,Retail,oil and fats,USD,L,...,2022,NaN,NaN,304.268105,104.705171,1266.996031,98.939324,NaN,3.0753,100.000000
2164811,ZWE,Zimbabwe,Zimuto Siding,-19.858553,30.859637,2022-07,Retail,pulses and nuts,USD,KG,...,2022,NaN,NaN,304.268105,104.705171,1266.996031,77.254819,NaN,2.4851,158.484568
2164812,ZWE,Zimbabwe,Zimuto Siding,-19.858553,30.859637,2022-11,Retail,pulses and nuts,USD,KG,...,2022,NaN,NaN,304.268105,104.705171,1266.996031,122.436966,NaN,2.4851,100.000000


<a id='basket'></a>
## Food basket size

In this context, food basket refers to the number of staple products included in the monitoring by the UN World Food Programme.

[Back to top](#top)

In [78]:
#Calculating the number of products that are monitored in each country

df_base_price['basket_size']=df_base_price.groupby('iso')['product_name'].transform('nunique')
df_base_price.head()

,iso,country,market,latitude,longitude,year_month,pricetype,category,currency,measure_unit,...,population,millions_undernourished,estim_earnings,inflation,gdp_pcapita,affordability_index,%_undernourished,base_price,price_index,basket_size
0,AFG,Afghanistan,Fayzabad,37.116638,70.580022,2000-03,Retail,non-food,AFN,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.5,126582.278481,10
1,AFG,Afghanistan,Fayzabad,37.116638,70.580022,2000-04,Retail,non-food,AFN,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.5,124050.632911,10
2,AFG,Afghanistan,Fayzabad,37.116638,70.580022,2000-05,Retail,non-food,AFN,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.5,112974.683544,10
3,AFG,Afghanistan,Fayzabad,37.116638,70.580022,2000-06,Retail,non-food,AFN,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.5,151392.405063,10
4,AFG,Afghanistan,Fayzabad,37.116638,70.580022,2000-07,Retail,non-food,AFN,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.5,158227.848101,10


In [79]:
df_base_price['basket_size'].describe()

count    2.164814e+06
mean     2.712531e+01
std      1.492462e+01
min      1.000000e+00
25%      1.400000e+01
50%      2.700000e+01
75%      3.600000e+01
max      5.700000e+01
Name: basket_size, dtype: float64

<a id='growth'></a>
## Annual Price Growth

This represents the percentage difference between the first and last recorded prices for a given product/country within one year.

[Back to top](#top)

In [80]:
# First, I want to isolate the records with the earliest (min) and latest (max) date for each country, year and product 

df_growth_max=df_full[['country',
                       'market',
                       'date',
                       'year',
                       'pricetype',
                       'product_name']].groupby(['country',
                                                 'market',
                                                 'year',
                                                 'pricetype',
                                                 'product_name']).agg({'date':np.max})

In [81]:
df_growth_max.reset_index(inplace=True)
df_growth_max.head()

,country,market,year,pricetype,product_name,date
0,Afghanistan,Badakhshan,2020,Retail,Bread,2020-12-01
1,Afghanistan,Badakhshan,2020,Retail,Diesel,2020-12-01
2,Afghanistan,Badakhshan,2020,Retail,Oil,2020-12-01
3,Afghanistan,Badakhshan,2020,Retail,Pulses,2020-12-01
4,Afghanistan,Badakhshan,2020,Retail,Rice,2020-12-01


In [82]:
df_growth_max.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270956 entries, 0 to 270955
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   country       270956 non-null  object        
 1   market        270956 non-null  object        
 2   year          270956 non-null  int64         
 3   pricetype     270956 non-null  object        
 4   product_name  270956 non-null  object        
 5   date          270956 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 12.4+ MB


In [83]:
df_growth_min=df_full[['country',
                       'market',
                       'date',
                       'year',
                       'pricetype',
                       'product_name']].groupby(['country',
                                                 'market',
                                                 'year',
                                                 'pricetype',
                                                 'product_name']).agg({'date':np.min})

In [84]:
df_growth_min.reset_index(inplace=True)
df_growth_min.head()

,country,market,year,pricetype,product_name,date
0,Afghanistan,Badakhshan,2020,Retail,Bread,2020-05-01
1,Afghanistan,Badakhshan,2020,Retail,Diesel,2020-05-01
2,Afghanistan,Badakhshan,2020,Retail,Oil,2020-05-01
3,Afghanistan,Badakhshan,2020,Retail,Pulses,2020-05-01
4,Afghanistan,Badakhshan,2020,Retail,Rice,2020-05-01


In [85]:
df_growth_min.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270956 entries, 0 to 270955
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   country       270956 non-null  object        
 1   market        270956 non-null  object        
 2   year          270956 non-null  int64         
 3   pricetype     270956 non-null  object        
 4   product_name  270956 non-null  object        
 5   date          270956 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 12.4+ MB


In [86]:
#Now I can add prices min_date_price and max_date_price to the dataframes

df_max_price=pd.merge(df_growth_max, df_full[['country',
                                              'market',
                                                   'date',
                                              'year',
                                              'pricetype',
                                                   'product_name',
                                                   'price_unit']], on=['country',
                                                                       'market',
                                                                       'date',
                                                                       'year',
                                                                       'pricetype',
                                                                       'product_name'])

In [87]:
df_max_price.head()

,country,market,year,pricetype,product_name,date,price_unit
0,Afghanistan,Badakhshan,2020,Retail,Bread,2020-12-01,50.00
1,Afghanistan,Badakhshan,2020,Retail,Diesel,2020-12-01,40.00
2,Afghanistan,Badakhshan,2020,Retail,Oil,2020-12-01,117.52
3,Afghanistan,Badakhshan,2020,Retail,Pulses,2020-12-01,78.00
4,Afghanistan,Badakhshan,2020,Retail,Rice,2020-12-01,76.74


In [88]:
df_max_price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270956 entries, 0 to 270955
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   country       270956 non-null  object        
 1   market        270956 non-null  object        
 2   year          270956 non-null  int64         
 3   pricetype     270956 non-null  object        
 4   product_name  270956 non-null  object        
 5   date          270956 non-null  datetime64[ns]
 6   price_unit    270956 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 16.5+ MB


In [89]:
df_max_price.rename(columns={'price_unit':'max_price'}, inplace=True)

In [90]:
df_min_price=pd.merge(df_growth_min, df_full[['country',
                                              'market',
                                                   'date',
                                              'year',
                                              'pricetype',
                                                   'product_name',
                                                   'price_unit']], on=['country',
                                                                       'market',
                                                                       'date',
                                                                       'year',
                                                                       'pricetype',
                                                                       'product_name'])
df_min_price.head()

,country,market,year,pricetype,product_name,date,price_unit
0,Afghanistan,Badakhshan,2020,Retail,Bread,2020-05-01,50.00
1,Afghanistan,Badakhshan,2020,Retail,Diesel,2020-05-01,39.25
2,Afghanistan,Badakhshan,2020,Retail,Oil,2020-05-01,100.00
3,Afghanistan,Badakhshan,2020,Retail,Pulses,2020-05-01,91.00
4,Afghanistan,Badakhshan,2020,Retail,Rice,2020-05-01,82.75


In [91]:
df_min_price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270956 entries, 0 to 270955
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   country       270956 non-null  object        
 1   market        270956 non-null  object        
 2   year          270956 non-null  int64         
 3   pricetype     270956 non-null  object        
 4   product_name  270956 non-null  object        
 5   date          270956 non-null  datetime64[ns]
 6   price_unit    270956 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 16.5+ MB


In [92]:
df_min_price.rename(columns={'price_unit':'min_price'},  inplace=True)

In [93]:
df_growth_dif=pd.merge(df_max_price[['country','market','year','pricetype','product_name','max_price']],
                       df_min_price[['country','market','year','pricetype','product_name','min_price']], on=['country','market','year','pricetype','product_name'])
df_growth_dif.head()

,country,market,year,pricetype,product_name,max_price,min_price
0,Afghanistan,Badakhshan,2020,Retail,Bread,50.00,50.00
1,Afghanistan,Badakhshan,2020,Retail,Diesel,40.00,39.25
2,Afghanistan,Badakhshan,2020,Retail,Oil,117.52,100.00
3,Afghanistan,Badakhshan,2020,Retail,Pulses,78.00,91.00
4,Afghanistan,Badakhshan,2020,Retail,Rice,76.74,82.75


In [94]:
df_growth_dif.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270956 entries, 0 to 270955
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   country       270956 non-null  object 
 1   market        270956 non-null  object 
 2   year          270956 non-null  int64  
 3   pricetype     270956 non-null  object 
 4   product_name  270956 non-null  object 
 5   max_price     270956 non-null  float64
 6   min_price     270956 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 16.5+ MB


In [95]:
# Calculating % anual growth

df_growth_dif['%_annual_growth']=round((df_growth_dif['max_price']-df_growth_dif['min_price'])/df_growth_dif['min_price']*100, 2)
df_growth_dif.head(30) #though some % are negative, in general the numbers look realistic

,country,market,year,pricetype,product_name,max_price,min_price,%_annual_growth
0,Afghanistan,Badakhshan,2020,Retail,Bread,50.000,50.000,0.00
1,Afghanistan,Badakhshan,2020,Retail,Diesel,40.000,39.250,1.91
2,Afghanistan,Badakhshan,2020,Retail,Oil,117.520,100.000,17.52
3,Afghanistan,Badakhshan,2020,Retail,Pulses,78.000,91.000,-14.29
4,Afghanistan,Badakhshan,2020,Retail,Rice,76.740,82.750,-7.26
5,Afghanistan,Badakhshan,2020,Retail,Salt,25.000,25.000,0.00
6,Afghanistan,Badakhshan,2020,Retail,Sugar,50.000,50.000,0.00
7,Afghanistan,Badakhshan,2020,Retail,Wheat,30.000,31.000,-3.23
8,Afghanistan,Badakhshan,2020,Retail,Wheat flour,37.260,39.500,-5.67
9,Afghanistan,Badakhshan,2021,Retail,Bread,72.450,50.000,44.90


In [96]:
# Now I can add the annual growth % to the main dataframe

df_merged=pd.merge(df_base_price,df_growth_dif[['country',
                                                'market',
                                               'year',
                                                'pricetype',
                                               'product_name',
                                               '%_annual_growth']], on=['country',
                                                                        'market',
                                                                        'year',
                                                                        'pricetype',
                                                                        'product_name'], how='left')

In [97]:
df_merged.tail(50)

,iso,country,market,latitude,longitude,year_month,pricetype,category,currency,measure_unit,...,millions_undernourished,estim_earnings,inflation,gdp_pcapita,affordability_index,%_undernourished,base_price,price_index,basket_size,%_annual_growth
2164764,ZWE,Zimbabwe,Selonga Business Centre,-21.384210,29.173125,2022-12,Retail,miscellaneous food,USD,KG,...,NaN,304.268105,104.705171,1266.996031,229.048558,NaN,1.3284,100.000000,21,0.00
2164765,ZWE,Zimbabwe,Selonga Business Centre,-21.384210,29.173125,2022-12,Retail,non-food,USD,KG,...,NaN,304.268105,104.705171,1266.996031,84.275455,NaN,3.6104,100.000000,21,0.00
2164766,ZWE,Zimbabwe,Selonga Business Centre,-21.384210,29.173125,2022-12,Retail,non-food,USD,KG,...,NaN,304.268105,104.705171,1266.996031,131.189628,NaN,2.3193,100.000000,21,0.00
2164767,ZWE,Zimbabwe,Selonga Business Centre,-21.384210,29.173125,2022-12,Retail,non-food,USD,L,...,NaN,304.268105,104.705171,1266.996031,25.451117,NaN,11.9550,100.000000,21,0.00
2164768,ZWE,Zimbabwe,Selonga Business Centre,-21.384210,29.173125,2022-12,Retail,oil and fats,USD,L,...,NaN,304.268105,104.705171,1266.996031,91.630460,NaN,3.3206,100.000000,21,0.00
2164769,ZWE,Zimbabwe,Selonga Business Centre,-21.384210,29.173125,2022-12,Retail,pulses and nuts,USD,KG,...,NaN,304.268105,104.705171,1266.996031,127.250263,NaN,2.3911,100.000000,21,0.00
2164770,ZWE,Zimbabwe,Sipambi,-20.327229,30.726516,2022-07,Retail,cereals and tubers,USD,KG,...,NaN,304.268105,104.705171,1266.996031,183.128561,NaN,1.6615,100.000000,21,0.00
2164771,ZWE,Zimbabwe,Sipambi,-20.327229,30.726516,2022-07,Retail,"meat, fish and eggs",USD,KG,...,NaN,304.268105,104.705171,1266.996031,17.658283,NaN,17.2309,100.000000,21,0.00
2164772,ZWE,Zimbabwe,Sipambi,-20.327229,30.726516,2022-07,Retail,miscellaneous food,USD,KG,...,NaN,304.268105,104.705171,1266.996031,329.615540,NaN,0.9231,100.000000,21,0.00
2164773,ZWE,Zimbabwe,Sipambi,-20.327229,30.726516,2022-07,Retail,miscellaneous food,USD,KG,...,NaN,304.268105,104.705171,1266.996031,141.263803,NaN,2.1539,100.000000,21,0.00


In [98]:
df_merged['%_annual_growth'].isnull().sum() #all 

0

In [99]:
df_merged.info() #number of records remained the same

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2164814 entries, 0 to 2164813
Data columns (total 26 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   iso                      object        
 1   country                  object        
 2   market                   object        
 3   latitude                 float64       
 4   longitude                float64       
 5   year_month               object        
 6   pricetype                object        
 7   category                 object        
 8   currency                 object        
 9   measure_unit             object        
 10  product_name             object        
 11  price_unit               float64       
 12  usdprice_unit            float64       
 13  date                     datetime64[ns]
 14  year                     int32         
 15  population               float64       
 16  millions_undernourished  float64       
 17  estim_earnings           fl

In [100]:
df_merged.duplicated().value_counts()

False    2164814
dtype: int64

In [101]:
df_merged['product_name'].replace('Shrimp','Shrimps', inplace=True)

In [102]:
df_merged[df_merged['product_name']=='Shrimps']['country'].value_counts()

Philippines    1676
Gambia, The     637
Benin           465
Name: country, dtype: int64

In [103]:
# Removing columns that I won't need

df_for_tableau=df_merged.drop(columns=['base_price','year','year_month'])

In [104]:
# Exporting data for further analysis. Final dataframe: 2164814 records, 21 columns

df_merged.to_pickle(os.path.join(path,'market_level_data_final.pkl'))
df_for_tableau.to_csv(os.path.join(path,'market_level_data_final.csv'))